In [1]:
# Natural Language Processing

# Importing the libraries
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
from sklearn.feature_extraction.text import CountVectorizer
import lightgbm as lgbm
import re


In [2]:
train=pd.read_csv('twitter_train.csv')
test=pd.read_csv('twitter_test.csv')

In [3]:
train.head()

,ID,text,target
0,train_0,The bitcoin halving is cancelled due to,1
1,train_1,MercyOfAllah In good times wrapped in its gran...,0
2,train_2,266 Days No Digital India No Murder of e learn...,1
3,train_3,India is likely to run out of the remaining RN...,1
4,train_4,In these tough times the best way to grow is t...,0


In [4]:
test.head()

,ID,text
0,test_2,Why is explained in the video take a look
1,test_3,Ed Davey fasting for Ramadan No contest
2,test_4,Is Doja Cat good or do you just miss Nicki Minaj
3,test_8,How Boris Johnson s cheery wounded in action p...
4,test_9,Man it s terrible Not even a reason to get on ...


In [5]:
import re
import nltk
from nltk.corpus import stopwords
from nltk.stem.porter import PorterStemmer


In [6]:
# PEROFRMING DATA CLEANING, word segmentation AND STREMMING
corpus2 = []
for i in range(0, len(train['text'])):
    new_test2 = re.sub('[^a-zA-Z]', ' ', train['text'][i])
    new_test2 = new_test2.lower()
    new_test2 = new_test2.split()
    ps = PorterStemmer()
    new_test2 = [ps.stem(word) for word in new_test2 if not word in set(stopwords.words('english'))]
    new_test2 = ' '.join(new_test2)
    corpus2.append(new_test2)

In [7]:
import nltk


In [8]:
# performing tokenization
wordfreq = {}
for sentence in corpus2:
    tokens = nltk.word_tokenize(sentence)
    for token in tokens:
        if token not in wordfreq.keys():
            wordfreq[token] = 1
        else:
            wordfreq[token] += 1

In [9]:
import heapq
most_freq = heapq.nlargest(3000, wordfreq, key=wordfreq.get)

In [10]:
# most_freq

In [11]:
# creating a bag of words from the most frequent words in the dataset
sentence_vectors = []
for sentence in corpus2:
    sentence_tokens = nltk.word_tokenize(sentence)
    sent_vec = []
    for token in most_freq:
        if token in sentence_tokens:
            sent_vec.append(1)
        else:
            sent_vec.append(0)
    sentence_vectors.append(sent_vec)

In [12]:
 sentence_vectors = np.asarray(sentence_vectors)


In [13]:
sentence_vectors 

array([[0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 1, ..., 0, 0, 0],
       ...,
       [0, 1, 0, ..., 0, 0, 0],
       [1, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0]])

In [14]:
del train['ID']

In [15]:
X=sentence_vectors 
y = train.iloc[:, 1].values
# X=np.clip(X,0.009,0.99)

In [16]:
import lightgbm as lgbm
import re

params = {
    'objective' :'binary',
    'learning_rate' : 0.01,
    'num_leaves' : 70,
    'feature_fraction': 0.6, 
    'bagging_fraction': 0.9, 
    'bagging_freq':0,
    'max_depth':-1,
    'min_data_in_leaf':0,
    'boosting_type' : 'gbdt',
    'metric': 'binary_logloss'
}

In [17]:
    from sklearn.model_selection import train_test_split
    # making test - valid sets
    X_train, X_test, Y_train, Y_test = train_test_split(X,y, random_state=0, test_size=0.25)
    
    # making lgbm datasets for train and valid
    d_train = lgbm.Dataset(X_train, Y_train)
    d_valid = lgbm.Dataset(X_test, Y_test)
    
    # training with early stop
    bst = lgbm.train(params, d_train, 5000, valid_sets=[d_valid], verbose_eval=200, early_stopping_rounds=2000)
    
    # making prediciton for one column

    y_pred = bst.predict(X_test)
#     y_pred=np.clip(y_pred,0.009,0.99)
print('Fininshed Training')

Training until validation scores don't improve for 2000 rounds
[200]	valid_0's binary_logloss: 0.301349
[400]	valid_0's binary_logloss: 0.229903
[600]	valid_0's binary_logloss: 0.210505
[800]	valid_0's binary_logloss: 0.207114
[1000]	valid_0's binary_logloss: 0.209078
[1200]	valid_0's binary_logloss: 0.213281
[1400]	valid_0's binary_logloss: 0.220292
[1600]	valid_0's binary_logloss: 0.228345
[1800]	valid_0's binary_logloss: 0.236744
[2000]	valid_0's binary_logloss: 0.245456
[2200]	valid_0's binary_logloss: 0.254862
[2400]	valid_0's binary_logloss: 0.264231
[2600]	valid_0's binary_logloss: 0.274202
Early stopping, best iteration is:
[756]	valid_0's binary_logloss: 0.206826
Fininshed Training


In [18]:
test

,ID,text
0,test_2,Why is explained in the video take a look
1,test_3,Ed Davey fasting for Ramadan No contest
2,test_4,Is Doja Cat good or do you just miss Nicki Minaj
3,test_8,How Boris Johnson s cheery wounded in action p...
4,test_9,Man it s terrible Not even a reason to get on ...
...,...,...
1957,test_2932,Fageeru meehaa geyga Bandah PUBLIC fundS amp G...
1958,test_2934,DFFN Diffusion Pharmaceuticals Announces Pre I...
1959,test_2936,I want to wish the Muslim members of Congress ...
1960,test_2937,You mean you don t believe there is a conspira...


In [19]:
ID_test= test['ID']

In [20]:
del test['ID']

In [21]:
corpus3 = []
for i in range(0, len(test['text'])):
    new_test3 = re.sub('[^a-zA-Z]', ' ', test['text'][i])
    new_test3 = new_test3.lower()
    new_test3 = new_test3.split()
    ps = PorterStemmer()
    new_test3 = [ps.stem(word) for word in new_test3 if not word in set(stopwords.words('english'))]
    new_test3 = ' '.join(new_test3)
    corpus3.append(new_test3)

In [22]:
wordfreq2 = {}
for sentence in corpus3:
    tokens = nltk.word_tokenize(sentence)
    for token in tokens:
        if token not in wordfreq2.keys():
            wordfreq2[token] = 1
        else:
            wordfreq2[token] += 1

In [23]:
import heapq
most_freq2 = heapq.nlargest(3500, wordfreq2, key=wordfreq2.get)

In [24]:
# most_freq2

In [25]:
sentence_vectors2 = []
for sentence in corpus3:
    sentence_tokens = nltk.word_tokenize(sentence)
    sent_vec = []
    for token in most_freq:
        if token in sentence_tokens:
            sent_vec.append(1)
        else:
            sent_vec.append(0)
    sentence_vectors2.append(sent_vec)

In [26]:
sentence_vectors2 = np.asarray(sentence_vectors2)

In [27]:
X2=sentence_vectors2

# X2=np.clip(X2,0.1,0.6)

In [28]:
Final_target = bst.predict(X2)


In [29]:
submission_Twitter_zindi = pd.DataFrame({"ID":ID_test,
    "target":Final_target })
submission_Twitter_zindi.to_csv("subTwitter_z1.csv", index=False)